In [1]:
import os, time, copy
import numpy as np
import torch
import torch.nn as nn
import torch.optim as optim
import torch.backends.cudnn as cudnn
import torch.utils.data as data
import torchvision
import torchvision.transforms as transforms
import torchvision.models as models
import torchvision.datasets as datasets
from PIL import Image

from train import *
from utils import *
from dataset import *
from metrics import *
from FCN.fcn8 import *

In [2]:
set_seed(1)
os.environ["CUDA_VISIBLE_DEVICES"] = '7'


In [3]:
# Load data
train_data = VOC(mode = 'train', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
train_loader = data.DataLoader(train_data, batch_size=1, num_workers = 0, shuffle=True)

val_data = VOC(mode = 'val', transform=transforms.ToTensor(), target_transform=transforms.ToTensor())
val_loader = data.DataLoader(val_data, batch_size=1, num_workers = 0, shuffle=True)

In [4]:
def train(model, device, train_loader, criterion, optimizer):
    model.train()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0

    loss_sum = torch.zeros(1)
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    optimizer.zero_grad()
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
            
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc += pixel_accuracy(out_mask, target_mask)/20
        macc += mean_accuracy(out_mask, target_mask)/20
        mIU += mean_IU(out_mask, target_mask)/20
        fIU += frequency_weighted_IU(out_mask, target_mask)/20
        
        #calculate loss
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)/20
        loss.backward()
        loss_sum += loss
        
        # Implement mini-batch by accumulate 
        # update gradient every 20 image
        if i%20 == 19 or i == 1111:
            optimizer.step()
            print("Batch %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i//20, loss_sum, pacc, macc, mIU, fIU))
            
            running_loss += loss_sum.item() * 20
            r_pacc += pacc * 20
            r_macc += macc * 20
            r_mIU += mIU * 20
            r_fIU += fIU * 20
            
            # batch initialize
            loss_sum, pacc, macc, mIU, fIU = 0, 0, 0, 0, 0
            optimizer.zero_grad()
            
            # show output mask
#             img = decode_segmap(out_mask)
#             plt.imshow(img); plt.axis('off'); plt.show()
        
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1112 for m in epoch_metric]
    
    print('Train:\t%s' %epoch_metric)
    return epoch_metric

In [5]:
def validate(model, device, val_loader, criterion, mode = 'val'):
    # switch to evaluate mode
    model.eval()

    running_loss = 0.0
    r_pacc, r_macc, r_mIU, r_fIU = 0, 0, 0, 0
    pacc, macc, mIU, fIU = 0, 0, 0, 0
    
    for i, (image, label) in enumerate(train_loader):
        if device == 'cuda':
            image, label = image.to(device), label.to(device)
        output = model(image)
        
        #calculate metrics
        out_mask = torch.argmax(output.cpu().squeeze(), dim=0).numpy()
        target_mask = label.cpu().squeeze()
        pacc = pixel_accuracy(out_mask, target_mask)
        macc = mean_accuracy(out_mask, target_mask)
        mIU = mean_IU(out_mask, target_mask)
        fIU = frequency_weighted_IU(out_mask, target_mask)
        
        o = torch.squeeze(output, 0).reshape(21, -1).permute(1, 0)
        l = torch.squeeze(label.reshape(1, -1).permute(1, 0), 1)
        loss = criterion(o, l)
        
        if i % 100 == 99:
            print("Val %d)\t%.3f\t%.3f\t%.3f\t%.3f\t%.3f" %(i, loss, pacc, macc, mIU, fIU))
        # statistics
        running_loss += loss.item()
        r_pacc += pacc
        r_macc += macc
        r_mIU += mIU
        r_fIU += fIU

    #epoch_loss = running_loss / 1111
    epoch_metric = (running_loss, r_pacc, r_macc, r_mIU, r_fIU)
    epoch_metric = [m/1111 for m in epoch_metric]
    
    if mode == 'val':
        print('Val metric:\t%s' %epoch_metric)
    else:
        print('Test metric:\t%s' %epoch_metric)
    
    return epoch_metric  #, copy.deepcopy(model.state_dict())

In [6]:
def train_model(model, device, train_loader, val_loader, lr = 1e-4, dict_name = 'fcn32', num_epochs=50):
    
    criterion = nn.CrossEntropyLoss(ignore_index = 255)
    optimizer = optim.SGD(model.parameters(), lr=lr, momentum=0.9, weight_decay=5**(-4))

    if device == 'cuda':
        model.to(device)
        cudnn.benchmark = True
        criterion = criterion.cuda()
    
    since = time.time()
    #M = {'train' : [], 'val' : []}
    
    best_model_wts = copy.deepcopy(model.state_dict())
    best_acc = 0.0

    for epoch in range(num_epochs):
        print('Epoch {}/{}'.format(epoch, num_epochs - 1))
        print('-' * 10)
        
        
        train_metric = train(model, device, train_loader, criterion, optimizer)
        M['train'].append(train_metric)
        
        # evaluate on validation set
        val_metric = validate(model, device, val_loader, criterion)
        M['val'].append(val_metric)
        
        if val_metric[1] > best_acc:
            best_acc = val_metric[1]
            best_model_wts = copy.deepcopy(model.state_dict())
            SaveDict(best_model_wts, 'fcn8_best')
        
        if epoch % 20 == 19:
            SaveDict(model.state_dict(), dict_name+'_epoch'+str(epoch))
        print()
    
    print()
    time_elapsed = time.time() - since
    print('Training complete in {:.0f}m {:.0f}s'.format(
        time_elapsed // 60, time_elapsed % 60))
    print('Best val Acc: {:4f}'.format(best_acc))

    # load best model weights
    print(model.load_state_dict(best_model_wts))
    return best_model_wts

In [7]:
global M
M = {'train' : [], 'val' : []}

device = 'cuda' if torch.cuda.is_available() else 'cpu'

fcn8 = FCN8()

# Initialize with fcn32
fcn32dict = LoadDict('fcn32_best')
del fcn32dict['up_final.weight']
fcn8.load_state_dict(fcn32dict, strict = False)

# Train
train_model(fcn8, device, train_loader, val_loader, lr = 1e-6, 
                           dict_name = 'fcn8', num_epochs=200)

#SaveDict(fcn8_best, 'fcn8_best')
SaveDict(M, 'fcn8_metric')

Epoch 0/199
----------
Batch 0)	0.203	0.930	0.744	0.651	0.808
Batch 1)	0.215	0.922	0.813	0.682	0.751
Batch 2)	0.225	0.921	0.648	0.564	0.793
Batch 3)	0.212	0.921	0.693	0.594	0.783
Batch 4)	0.170	0.935	0.694	0.601	0.816
Batch 5)	0.195	0.927	0.779	0.682	0.769
Batch 6)	0.203	0.924	0.809	0.696	0.754
Batch 7)	0.176	0.933	0.759	0.655	0.791
Batch 8)	0.188	0.932	0.756	0.661	0.787
Batch 9)	0.184	0.929	0.796	0.688	0.812
Batch 10)	0.227	0.918	0.714	0.610	0.770
Batch 11)	0.196	0.926	0.772	0.676	0.790
Batch 12)	0.204	0.927	0.759	0.661	0.793
Batch 13)	0.220	0.920	0.683	0.571	0.747
Batch 14)	0.197	0.925	0.857	0.735	0.779
Batch 15)	0.218	0.918	0.807	0.710	0.795
Batch 16)	0.236	0.914	0.781	0.649	0.744
Batch 17)	0.184	0.933	0.781	0.683	0.814
Batch 18)	0.197	0.926	0.773	0.659	0.772
Batch 19)	0.182	0.939	0.758	0.668	0.831
Batch 20)	0.257	0.909	0.781	0.668	0.757
Batch 21)	0.181	0.932	0.798	0.702	0.815
Batch 22)	0.242	0.912	0.680	0.577	0.756
Batch 23)	0.208	0.926	0.838	0.735	0.803
Batch 24)	0.221	0.917	0.820

Val 199)	0.134	0.946	0.937	0.794	0.864
Val 299)	0.330	0.863	0.803	0.653	0.672
Val 399)	0.226	0.916	0.920	0.825	0.805
Val 499)	0.161	0.945	0.965	0.743	0.800
Val 599)	0.169	0.944	0.605	0.483	0.870
Val 699)	0.276	0.891	0.883	0.772	0.742
Val 799)	0.077	0.969	0.914	0.768	0.908
Val 899)	0.242	0.907	0.908	0.765	0.779
Val 999)	0.140	0.940	0.788	0.603	0.839
Val 1099)	0.101	0.959	0.714	0.650	0.914
Val metric:	[0.18855635475192947, 0.9310631508551092, 0.820470432655524, 0.6850416440904074, 0.8034745478589564]

Epoch 3/199
----------
Batch 0)	0.161	0.938	0.745	0.634	0.826
Batch 1)	0.195	0.929	0.779	0.683	0.819
Batch 2)	0.262	0.903	0.832	0.710	0.745
Batch 3)	0.214	0.920	0.758	0.653	0.792
Batch 4)	0.227	0.917	0.797	0.683	0.782
Batch 5)	0.192	0.929	0.770	0.658	0.808
Batch 6)	0.244	0.907	0.726	0.623	0.770
Batch 7)	0.224	0.916	0.762	0.654	0.784
Batch 8)	0.194	0.927	0.724	0.626	0.806
Batch 9)	0.184	0.929	0.808	0.694	0.798
Batch 10)	0.241	0.909	0.751	0.632	0.756
Batch 11)	0.191	0.929	0.727	0.634	0.813
Ba

Batch 47)	0.158	0.944	0.807	0.703	0.835
Batch 48)	0.205	0.926	0.777	0.672	0.815
Batch 49)	0.176	0.935	0.783	0.667	0.810
Batch 50)	0.178	0.934	0.791	0.689	0.820
Batch 51)	0.217	0.918	0.805	0.670	0.754
Batch 52)	0.193	0.924	0.801	0.678	0.785
Batch 53)	0.185	0.933	0.732	0.639	0.827
Batch 54)	0.230	0.915	0.742	0.642	0.767
Batch 55)	0.120	0.554	0.517	0.445	0.482
Train:	[0.19360050063231865, 0.9283389610400627, 0.7801680970153649, 0.6703131627722043, 0.7968292990307018]
Val 99)	0.219	0.914	0.928	0.772	0.756
Val 199)	0.214	0.931	0.858	0.602	0.785
Val 299)	0.143	0.953	0.947	0.878	0.843
Val 399)	0.249	0.902	0.911	0.723	0.761
Val 499)	0.173	0.934	0.944	0.849	0.835
Val 599)	0.054	0.980	0.942	0.870	0.945
Val 699)	0.095	0.966	0.971	0.804	0.921
Val 799)	0.022	0.998	0.998	0.713	0.226
Val 899)	0.206	0.930	0.939	0.809	0.806
Val 999)	0.106	0.952	0.917	0.770	0.868
Val 1099)	0.087	0.985	0.844	0.809	0.959
Val metric:	[0.1881135320324454, 0.9313576198731028, 0.8223533841236612, 0.6859577729476831, 0.8040095

Batch 35)	0.176	0.935	0.769	0.671	0.809
Batch 36)	0.189	0.932	0.791	0.681	0.820
Batch 37)	0.168	0.939	0.774	0.673	0.833
Batch 38)	0.185	0.930	0.798	0.697	0.800
Batch 39)	0.163	0.941	0.821	0.723	0.830
Batch 40)	0.170	0.935	0.748	0.633	0.804
Batch 41)	0.179	0.932	0.799	0.690	0.818
Batch 42)	0.246	0.906	0.792	0.675	0.762
Batch 43)	0.143	0.944	0.820	0.723	0.848
Batch 44)	0.215	0.918	0.773	0.654	0.741
Batch 45)	0.211	0.923	0.753	0.630	0.792
Batch 46)	0.177	0.935	0.811	0.692	0.750
Batch 47)	0.229	0.915	0.784	0.659	0.753
Batch 48)	0.186	0.935	0.775	0.670	0.825
Batch 49)	0.193	0.930	0.780	0.641	0.750
Batch 50)	0.206	0.919	0.805	0.700	0.793
Batch 51)	0.183	0.934	0.785	0.662	0.790
Batch 52)	0.207	0.929	0.689	0.605	0.809
Batch 53)	0.163	0.937	0.794	0.679	0.799
Batch 54)	0.222	0.920	0.710	0.595	0.788
Batch 55)	0.075	0.573	0.476	0.423	0.521
Train:	[0.19072867876334157, 0.9292093917175926, 0.7792605308496942, 0.6707515212907186, 0.7979981735902572]
Val 99)	0.126	0.967	0.574	0.445	0.881
Val 199)	0.04

Batch 22)	0.193	0.932	0.818	0.681	0.772
Batch 23)	0.174	0.935	0.772	0.660	0.799
Batch 24)	0.212	0.924	0.801	0.698	0.802
Batch 25)	0.202	0.920	0.843	0.714	0.747
Batch 26)	0.153	0.944	0.803	0.687	0.805
Batch 27)	0.185	0.929	0.818	0.693	0.768
Batch 28)	0.193	0.932	0.708	0.615	0.821
Batch 29)	0.243	0.909	0.776	0.668	0.776
Batch 30)	0.190	0.930	0.762	0.666	0.815
Batch 31)	0.191	0.929	0.768	0.664	0.807
Batch 32)	0.141	0.948	0.780	0.646	0.788
Batch 33)	0.211	0.923	0.763	0.650	0.783
Batch 34)	0.156	0.944	0.801	0.714	0.842
Batch 35)	0.238	0.914	0.693	0.587	0.784
Batch 36)	0.180	0.931	0.799	0.651	0.746
Batch 37)	0.184	0.931	0.816	0.696	0.781
Batch 38)	0.237	0.907	0.732	0.613	0.753
Batch 39)	0.184	0.927	0.793	0.678	0.796
Batch 40)	0.171	0.940	0.785	0.683	0.804
Batch 41)	0.178	0.937	0.817	0.710	0.813
Batch 42)	0.192	0.927	0.838	0.726	0.792
Batch 43)	0.159	0.942	0.819	0.729	0.840
Batch 44)	0.205	0.922	0.767	0.657	0.792
Batch 45)	0.181	0.934	0.775	0.669	0.815
Batch 46)	0.193	0.926	0.824	0.715	0.804


...

Batch 46)	0.194	0.931	0.734	0.637	0.789
Batch 47)	0.179	0.934	0.774	0.684	0.825
Batch 48)	0.166	0.939	0.849	0.717	0.774
Batch 49)	0.193	0.927	0.793	0.682	0.772
Batch 50)	0.158	0.940	0.843	0.719	0.774
Batch 51)	0.177	0.934	0.786	0.671	0.811
Batch 52)	0.128	0.951	0.795	0.703	0.840
Batch 53)	0.222	0.920	0.774	0.666	0.780
Batch 54)	0.261	0.899	0.789	0.647	0.736
Batch 55)	0.103	0.564	0.498	0.440	0.490
Train:	[0.17622488008128653, 0.9349330771532534, 0.7923413167528518, 0.6855046152563055, 0.8064160078065638]
Val 99)	0.033	0.986	0.705	0.696	0.961
Val 199)	0.123	0.953	0.857	0.713	0.893
Val 299)	0.162	0.943	0.839	0.588	0.796
Val 399)	0.417	0.879	0.658	0.536	0.656
Val 499)	0.096	0.971	0.887	0.812	0.897
Val 599)	0.337	0.853	0.601	0.457	0.648
Val 699)	0.222	0.924	0.816	0.546	0.707
Val 799)	0.124	0.964	0.903	0.788	0.914
Val 899)	0.239	0.955	0.944	0.750	0.900
Val 999)	0.235	0.908	0.913	0.786	0.792
Val 1099)	0.120	0.943	0.934	0.785	0.807
Val metric:	[0.17250292347308205, 0.9380301991507937, 0.836143

Batch 33)	0.187	0.930	0.775	0.653	0.773
Batch 34)	0.174	0.935	0.760	0.669	0.819
Batch 35)	0.146	0.944	0.898	0.795	0.837
Batch 36)	0.214	0.924	0.741	0.653	0.798
Batch 37)	0.185	0.933	0.778	0.681	0.821
Batch 38)	0.154	0.942	0.793	0.679	0.799
Batch 39)	0.205	0.928	0.756	0.645	0.792
Batch 40)	0.161	0.942	0.854	0.738	0.778
Batch 41)	0.165	0.940	0.758	0.654	0.823
Batch 42)	0.220	0.919	0.796	0.692	0.791
Batch 43)	0.178	0.935	0.817	0.708	0.803
Batch 44)	0.211	0.921	0.805	0.691	0.775
Batch 45)	0.200	0.929	0.843	0.737	0.772
Batch 46)	0.177	0.933	0.804	0.690	0.793
Batch 47)	0.162	0.939	0.790	0.692	0.831
Batch 48)	0.138	0.951	0.792	0.707	0.835
Batch 49)	0.208	0.927	0.788	0.686	0.803
Batch 50)	0.193	0.929	0.749	0.653	0.822
Batch 51)	0.173	0.933	0.832	0.733	0.800
Batch 52)	0.199	0.927	0.784	0.676	0.778
Batch 53)	0.175	0.941	0.801	0.691	0.827
Batch 54)	0.171	0.937	0.768	0.675	0.833
Batch 55)	0.090	0.567	0.498	0.440	0.506
Train:	[0.17623752864787903, 0.9349666203610603, 0.7912007858022831, 0.684578356

Batch 20)	0.183	0.935	0.799	0.696	0.806
Batch 21)	0.180	0.933	0.782	0.680	0.817
Batch 22)	0.207	0.922	0.785	0.654	0.759
Batch 23)	0.175	0.940	0.763	0.672	0.839
Batch 24)	0.148	0.944	0.798	0.709	0.842
Batch 25)	0.197	0.929	0.758	0.649	0.761
Batch 26)	0.161	0.938	0.786	0.664	0.785
Batch 27)	0.166	0.938	0.736	0.645	0.816
Batch 28)	0.152	0.943	0.802	0.710	0.826
Batch 29)	0.178	0.935	0.822	0.679	0.774
Batch 30)	0.193	0.931	0.754	0.651	0.813
Batch 31)	0.171	0.938	0.737	0.627	0.807
Batch 32)	0.168	0.941	0.823	0.667	0.751
Batch 33)	0.193	0.932	0.786	0.687	0.808
Batch 34)	0.200	0.926	0.788	0.681	0.804
Batch 35)	0.185	0.934	0.807	0.703	0.810
Batch 36)	0.145	0.945	0.846	0.755	0.836
Batch 37)	0.188	0.930	0.772	0.664	0.798
Batch 38)	0.151	0.945	0.759	0.661	0.837
Batch 39)	0.187	0.932	0.808	0.683	0.774
Batch 40)	0.154	0.946	0.710	0.627	0.837
Batch 41)	0.138	0.951	0.804	0.719	0.861
Batch 42)	0.193	0.924	0.797	0.682	0.794
Batch 43)	0.154	0.943	0.795	0.703	0.832
Batch 44)	0.178	0.936	0.769	0.666	0.807


Batch 7)	0.149	0.947	0.825	0.727	0.841
Batch 8)	0.161	0.944	0.791	0.698	0.841
Batch 9)	0.231	0.915	0.762	0.648	0.783
Batch 10)	0.175	0.937	0.730	0.634	0.816
Batch 11)	0.205	0.930	0.743	0.627	0.779
Batch 12)	0.146	0.946	0.836	0.734	0.834
Batch 13)	0.151	0.945	0.822	0.696	0.800
Batch 14)	0.191	0.927	0.800	0.689	0.796
Batch 15)	0.168	0.936	0.849	0.733	0.829
Batch 16)	0.153	0.943	0.823	0.721	0.831
Batch 17)	0.157	0.942	0.849	0.720	0.809
Batch 18)	0.183	0.932	0.787	0.679	0.807
Batch 19)	0.213	0.918	0.749	0.642	0.773
Batch 20)	0.159	0.940	0.841	0.726	0.798
Batch 21)	0.149	0.947	0.802	0.713	0.848
Batch 22)	0.203	0.924	0.734	0.625	0.790
Batch 23)	0.159	0.938	0.849	0.743	0.828
Batch 24)	0.168	0.938	0.836	0.714	0.786
Batch 25)	0.200	0.925	0.767	0.653	0.787
Batch 26)	0.171	0.936	0.842	0.719	0.785
Batch 27)	0.192	0.928	0.785	0.686	0.803
Batch 28)	0.213	0.923	0.771	0.670	0.796
Batch 29)	0.149	0.948	0.789	0.693	0.831
Batch 30)	0.122	0.956	0.804	0.719	0.871
Batch 31)	0.180	0.938	0.761	0.666	0.826
Bat

KeyboardInterrupt: 

In [8]:
SaveDict(M, 'fcn8_metric')

0